In [ ]:
# LLM
# mixtral:8x22b-instruct
# llama3:70b-instruct-q8_0
# dolphin-llama3:70b-v2.9-q8_0
local_llm = 'dolphin-llama3:70b-v2.9-q8_0'

In [ ]:
'Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n```\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'

In [ ]:
import requests

def fetch_model_names(base_url):
    api_endpoint = f"{base_url}/api/tags"
    try:
        response = requests.get(api_endpoint)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        model_names = [model['name'] for model in data.get('models', [])]
        return model_names
    except requests.RequestException as e:
        return str(e)

# Example usage
base_url = "https://ollama.junzis.com"
model_names = fetch_model_names(base_url)
print(model_names)


In [ ]:
# mixtral:8x22b-instruct
# llama3:70b-instruct-q8_0
# dolphin-llama3:70b-v2.9-q8_0
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, tool, create_tool_calling_agent, create_openai_tools_agent
from langchain_experimental.llms.ollama_functions import OllamaFunctions

local_llm = 'llama3:70b-instruct-q8_0'
ollama_llm = OllamaFunctions(base_url='https://ollama.junzis.com',
                             model=local_llm, temperature=0.0)


ollama_llm.invoke('Hello, how are you?')

In [ ]:
from typing import Optional
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_openai import ChatOpenAI, OpenAI
from dotenv import load_dotenv, find_dotenv
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# take environment variables from .env so we can load OPENAI_API_KEY
load_dotenv(find_dotenv())

json_schema = {
    "title": "equation",
    "description": "information about an equation.",
    "type": "object",
    "properties": {
        "first_number": {"title": "First Number", "description": "first number in the equation", "type": "string"},
        "second_number": {"title": "Second Number", "description": "Second number in the equation", "type": "string"},
        "operation": {"title": "Operation", "description": "operation to perform. Options: +, -, *, /", "type": "string"},
    },
    "required": ["first_number", "second_number", "operation"],
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are extracting information in structured formats."),
        ("human",
         "Use the given format to extract information from the following input: {input}")
    ]
)

#model = OllamaFunctions(model='phi3:latest', format="json")
model = ChatOpenAI()

chain = create_structured_output_chain(
    json_schema, model, prompt, verbose=True)

chain.run('Here I need to check how much is 3243 divided by 3. (i should use a calculator for this)')

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
@tool
def calculator(equation: str):
    """Simple 2 number calculator"""
    parsed_equation = chain.run(equation)
    print(parsed_equation)
    num1 = float(parsed_equation['first_number'])
    num2 = float(parsed_equation['second_number'])
    operation = str(parsed_equation['operation'])
    if operation == '+':
        return str(num1 + num2)
    elif operation == '-':
        return str(num1 - num2)
    elif operation == '*':
        return str(num1 * num2)
    elif operation == '/':
        return str(num1 / num2)
    else:
        return "Invalid operation"



tools = [calculator, TavilySearchResults()]

In [ ]:
ollama_llm = ChatOllama(base_url='https://ollama.junzis.com',
                        model='dolphin-llama3:70b-v2.9-q8_0', temperature=0.0)
ollama_llm.invoke('Hello, how are you?')

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_json_chat_agent

agent_prompt = hub.pull("hwchase17/react-chat-json")


# ollama_llm = OllamaFunctions(base_url='https://ollama.junzis.com',
#                         model='dolphin-llama3:70b-v2.9-q8_0', temperature=0.0, format='json')
# ollama_llm = ChatOpenAI(base_url='https://ollama.junzis.com/v1',
#                         model='dolphin-llama3:70b-v2.9-q8_0', temperature=0.0, api_key='ollama')
ollama_llm = ChatOllama(base_url='http://ollama.junzis.com',
                        model='dolphin-llama3:70b-v2.9-q8_0', temperature=0.0)
agent = create_json_chat_agent(ollama_llm, tools, agent_prompt)


agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.invoke({"input": "search how much an elephant weighs in pounds and then multiply it by 146376 to get the answer1, then search how much does a an A320 airplane weight and divide it by 20 to get answer2. Finally sum up answer1 and answer2 to get the answer."})

In [ ]:
from operator import itemgetter
from typing import Dict, List, Union

from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(
            tool_call["args"])
    return tool_calls


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, exponentiate, add]
functions = [convert_to_openai_function(t) for t in tools]
ollama_with_tools = ollama_llm.bind_tools(
    tools=functions, function_call=functions)
tool_map = {tool.name: tool for tool in tools}

chain2 = ollama_with_tools | call_tools

In [ ]:
model = ollama_llm.bind_tools(
    tools=functions,
    
)
model.invoke("what is 2 + 8")

In [ ]:
chain2 = model | call_tools
chain2.invoke("what is 2 + 8")

In [ ]:
functions

In [ ]:
ollama_with_tools.invoke("What is 2 to the power of 3?")

In [ ]:
function_call = {t['name']: t for t in functions}
function_call

In [ ]:
functions

In [ ]:
ollama_with_tools

In [ ]:
chain2.invoke("What's 23 times 7")

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "jlvdoorn/whisper-large-v3-atco2-asr"
# model_id = "scy0208/whisper-aviation-base"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
from pydub import AudioSegment
import os


def split_mp3(path):
    # Load the audio file
    audio = AudioSegment.from_mp3(path)

    # Calculate the duration of the audio in seconds
    duration = len(audio) / 1000  # pydub uses milliseconds

    # Create a directory for the split files
    base_path, filename = os.path.split(path)
    file_stem, _ = os.path.splitext(filename)
    output_dir = os.path.join(base_path, file_stem)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Split the audio file into 10-second chunks
    for i in range(0, int(duration), 10):
        # Calculate end time for each chunk
        end_time = min((i + 10) * 1000, len(audio))
        chunk = audio[i * 1000:end_time]

        # Define the filename for each chunk
        chunk_filename = f"{file_stem}_part_{i//10 + 1}.mp3"
        chunk_path = os.path.join(output_dir, chunk_filename)

        # Export the chunk to a new file
        chunk.export(chunk_path, format="mp3")

    print(f"Files are saved in: {output_dir}")


# Example usage:
split_mp3("C:\\Users\\justa\\Downloads\\crashKSEE-Twr-Apr-29-2024-0100Z.mp3")

In [ ]:
# List all files in the directory
directory_path = "C:\\Users\\justa\\Downloads\\crashKSEE-Twr-Apr-29-2024-0100Z"
files = [f for f in os.listdir(directory_path) if f.endswith('.mp3')]

# Sort files to ensure correct order
files.sort()

# Loop over sorted files
for file in files:
    # Construct full file path
    file_path = os.path.join(directory_path, file)

    # Process the file through the pipeline

    result = pipe(file_path)

    # Print the transcribed text
    print(result['text'])